In [13]:
import csv
import zipfile
import random

!wget --no-check-certificate "https://github.com/andynosbig/moviele/archive/refs/heads/main.zip" -O "/tmp/main.zip"
zip_ = zipfile.ZipFile('/tmp/main.zip', 'r')
zip_.extractall('/tmp')
zip_.close()

filmsByTitle = {}
with open("/tmp/moviele-main/combined.tsv", 'r') as basics:
    for row in csv.DictReader(basics, delimiter='\t'):
        filmsByTitle[row['title']] = row
        filmsByTitle[row['title']].pop('id')
        
        filmsByTitle[row['title']]['genres'] = row['genres'].split(',')
        filmsByTitle[row['title']]['principals'] = row['principals'].split(',')

principalNames = {}
with open("/tmp/moviele-main/principals.tsv", 'r') as basics:
    for row in csv.DictReader(basics, delimiter='\t'):
        principalNames[row['id']] = row['name']

--2022-02-19 23:39:27--  https://github.com/andynosbig/moviele/archive/refs/heads/main.zip
Resolving github.com (github.com)... 81.99.162.48, 2a02:8801:200:1::1
Connecting to github.com (github.com)|81.99.162.48|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://codeload.github.com/andynosbig/moviele/zip/refs/heads/main [following]
--2022-02-19 23:39:28--  https://codeload.github.com/andynosbig/moviele/zip/refs/heads/main
Resolving codeload.github.com (codeload.github.com)... 140.82.121.9
Connecting to codeload.github.com (codeload.github.com)|140.82.121.9|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 81312 (79K) [application/zip]
Saving to: ‘/tmp/main.zip’

/tmp/main.zip       100%[===================>]  79.41K  --.-KB/s    in 0.08s   

2022-02-19 23:39:28 (1.02 MB/s) - ‘/tmp/main.zip’ saved [81312/81312]



In [26]:
from ipywidgets import interact, interactive, fixed, interact_manual
from IPython.display import display
import ipywidgets as widgets

class Target:
    film = None
    guesses = 0
    
def get_random_item(list_):
    return list(list_)[random.randint(0, len(list_)-1)]

def calc_list_similarity(listA, listB):
    return 100 * (len(set(listA)&set(listB)) / len(set(listA)|set(listB))), list(set(listA)&set(listB))
    
def run():
    new_target_button = widgets.Button(description = 'Reset Target Film')
    get_answer_button = widgets.Button(description = 'Show Answer')
    
    out = widgets.Output(layout={'border': '1px solid black'})
    target = Target()
    def set_new_target(event):
        target.film = get_random_item(filmsByTitle.values())
        target.guesses = 0
        out.clear_output()
        with out:
            print(f"hint,  {principalNames[get_random_item(target.film['principals'])]} is involved in this film")

    set_new_target(None)
    new_target_button.on_click(set_new_target)

    def show_answer(event):
        with out:
            print("Answer: ", target.film['title'])

    get_answer_button.on_click(show_answer)
    display(widgets.HBox((new_target_button, get_answer_button)))   
    films = ["-"]
    films.extend(sorted(filmsByTitle.keys()))
    display(out)

    def test_guess(x):
        if x not in filmsByTitle:
            return

        if filmsByTitle[x] == target.film:
            return "Success!"
        target.guesses += 1
        guess_film = filmsByTitle[x]
        genre_percentage, genre_overlap = calc_list_similarity(guess_film['genres'], target.film['genres'])
        principals_percentage, principals_overlap = calc_list_similarity(guess_film['principals'], target.film['principals'])
        principal_name_overlap = [principalNames[x] for x in principals_overlap]
        year_diff = int(guess_film['startYear']) - int(target.film['startYear'])
        result = f"\n{target.guesses} {x} - \n\tprincipal matches\t{principals_percentage}% - {principal_name_overlap}  \n\tgenre matches\t\t{genre_percentage}% - {genre_overlap} - year difference {year_diff}"
        with out:
            print(result)

    interact(test_guess, x=films)
    interact


run()

Output(layout=Layout(border='1px solid black'))

interactive(children=(Dropdown(description='x', options=('-', '12 Monkeys', '12 Years a Slave', '13 Assassins'…